### Distill step by step finetuning approach - trying enhanced rationale with specific reasoning for date conversion

In [1]:
!sudo pip install -q transformers --upgrade

In [2]:
import transformers
transformers.__version__

'4.34.1'

In [3]:
#!sudo pip install -q accelerate peft==0.4.0 bitsandbytes trl==0.4.7

In [4]:
import os
import torch
from datasets import load_dataset
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import pandas as pd
import torch

In [37]:
df = pd.read_csv('/data/mistral/query-to-mql/exp-8/training-data.csv')

In [38]:
df.columns

Index(['Query', 'MQL', 'Rationale'], dtype='object')

In [39]:
df.shape

(65, 3)

In [5]:
# The model that you want to train from the Hugging Face hub
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# The instruction dataset to use
#dataset_name = ""

# Fine-tuned model name
#new_model = "mistral-ft-peft-on-template_and_user_query-data"

In [41]:
################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

In [42]:
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = True

In [74]:
################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "/data/mistral/query-to-mql/exp-8/oct-30"

# Number of training epochs
num_train_epochs = 15

# Enable fp16/bf16 training (set bf16 to True with an A100)
# fp16 = False
fp16 = True # not using quantisation
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 1

# Batch size per GPU for evaluation
per_device_eval_batch_size = 1

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True


# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = 200

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 40

# Log every X updates steps
logging_steps = 40

In [75]:
################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [45]:
promt_template = """Given the context : {context} and date reference: {date_input}, the query: {user_query}, is converted into below shown structured output.
[MQL]
{mql}
[/MQL]
the steps and rationale used to achieve above structured output is as below.
{rationale}
"""

In [46]:
df.columns

Index(['Query', 'MQL', 'Rationale'], dtype='object')

In [6]:
context = """{
    "MEASURE": [{"ENTITY": "Discount", "other names": ["discount", "discount rate", "discount value", "deduction"]},
                {"ENTITY": "Purchase Vol", "other names": ["purchase", "purchase value", "purchase model"]},
                {"ENTITY": "Quantity", "other names": ["quantity", "volume"]},
                {"ENTITY": "Sales", "other names": ["sales", "sale"]}],
    "DIMENSION": [{"ENTITY": "Sub-Category", "other names": ["sub-category", "sub category", "categories", "section"]},
                  {"ENTITY": "Segment", "other names": ["segment", "segments", "units", "divisions"]},
                  {"ENTITY": "Parts", "other names": ["parts", "part", "section", "divisions"]},
                  {"ENTITY": "Country", "other names": ["country", "countries"]}],
    "FILTER": [{"ENTITY": "Consumer", "other names": ["consumers", "consumer"], "parent": "Segment"},
               {"ENTITY": "Phone", "other names": ["phone", "phones", "mobile phones"], "parent": "Sub-Category"},
               {"ENTITY": "Binder", "other names": ["binders", "binder"], "parent": "Sub-Category"},
               {"ENTITY": "Corporate", "other names": ["corporates", "corporate"], "parent": "Segment"},
               {"ENTITY": "India", "other names": ["india"], "parent": "Country"},
               {"ENTITY": "Dubai", "other names": ["dubai"], "parent": "Country"}],
    "DERIVED MEASURE": [{"ENTITY": "Ratio",
             "other names": ["ratio", "share", "contribution", "percentage", "proportion", "contributing"]},
            {"ENTITY": "Why", "other names": ["why", "cause of", "reason for", "diagnose"]},
            {"ENTITY": "contribution_to_growth", "other names": ["contribution to growth", "growth", "grown"]},
            {"ENTITY": "kda_transactional", "other names": ["kda", "key drivers", "key driver", "drivers", "driver"]},
            {"ENTITY": "Growth Rate", "other names": ["growth rate", "growth", "grown"]},
            {"ENTITY": "correlation",
             "other names": ["associate", "associated", "association", "associations", "correlate", "correlated",
                             "correlation", "correlations", "relate", "related", "relation", "relations",
                             "relationship",
                             "relationships"]}
            ],
    "DATE VARIABLE": [{"ENTITY": "Order Date", "other names": ["order date", "date", "trend", "time", "when", "mom", "yoy"]}]
    }"""

In [7]:
date_input = {
    "start_date": "01/01/2020",
    "end_date": "15/09/2023"
}

In [49]:
def create_fine_tuning_dataset(row):
    mql = row['MQL']
    user_query = row['Query']
    rationale = row['Rationale']
    formated = promt_template.format(context=context,
                                             date_input=date_input,
                                             user_query=user_query,
                                             mql=mql,
                                             rationale=rationale)
    return formated

In [50]:
df['fine_tuning_dataset']=df.apply(create_fine_tuning_dataset, axis=1)

In [51]:
df.drop(columns=['Query', 'MQL', 'Rationale'], inplace=True)
df.shape

(65, 1)

In [52]:
train_dataset = Dataset.from_pandas(df)

In [53]:
train_dataset

Dataset({
    features: ['fine_tuning_dataset'],
    num_rows: 65
})

In [54]:
train_dataset['fine_tuning_dataset'][0]

'Given the context : {\n    "MEASURE": [{"ENTITY": "Discount", "other names": ["discount", "discount rate", "discount value", "deduction"]},\n                {"ENTITY": "Purchase Vol", "other names": ["purchase", "purchase value", "purchase model"]},\n                {"ENTITY": "Quantity", "other names": ["quantity", "volume"]},\n                {"ENTITY": "Sales", "other names": ["sales", "sale"]}],\n    "DIMENSION": [{"ENTITY": "Sub-Category", "other names": ["sub-category", "sub category", "categories", "section"]},\n                  {"ENTITY": "Segment", "other names": ["segment", "segments", "units", "divisions"]},\n                  {"ENTITY": "Parts", "other names": ["parts", "part", "section", "divisions"]},\n                  {"ENTITY": "Country", "other names": ["country", "countries"]}],\n    "FILTER": [{"ENTITY": "Consumer", "other names": ["consumers", "consumer"], "parent": "Segment"},\n               {"ENTITY": "Phone", "other names": ["phone", "phones", "mobile phones"

In [55]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [56]:
bnb_4bit_quant_type

'nf4'

In [57]:
compute_dtype

torch.float16

In [58]:
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

In [9]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

In [10]:
!sudo pip install -q pynvml

In [11]:
from pynvml.smi import nvidia_smi
nvsmi = nvidia_smi.getInstance()
nvsmi.DeviceQuery('memory.free, memory.total')

{'gpu': [{'fb_memory_usage': {'total': 16384.0,
    'free': 15972.9375,
    'unit': 'MiB'}}]}

In [62]:
#!df -H

In [12]:
torch.cuda.is_available()

True

In [64]:
# del model
# torch.cuda.empty_cache()

In [65]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
#     torch_dtype=torch.bfloat16,
    device_map="auto"
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True,
                                          # add_eos_token=True,
                                          use_fast=False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [67]:
max([len(tokenizer.encode(df['fine_tuning_dataset'][i])) for i in range(df.shape[0])])

1709

In [68]:
# LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules = ["q_proj", "v_proj"],
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

In [76]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
#     eval_steps=50, # requires when eval_dataset is defined
#     per_device_eval_batch_size=1, # Batch size for evaluation
#     evaluation_strategy="steps", # requires when eval_dataset is defined
    logging_strategy="steps",
    logging_steps=5,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=2000,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
#     load_best_model_at_end=True,
#     save_total_limit=1,
#     metric_for_best_model="eval_loss",
#     greater_is_better=False
)

In [77]:
## Getting FLOPs of model

model_flops = (
  model.floating_point_ops(
    {
       "input_ids": torch.zeros(
           (1, 2048)
      )
    }
  )
  * training_arguments.gradient_accumulation_steps
)

#print(model)
print("Memory footprint", model.get_memory_footprint() / 1e9, "GB")
print("Flops", model_flops / 1e9, "GFLOPs")

Memory footprint 5.185232896 GB
Flops 87710.798708736 GFLOPs


In [71]:
train_dataset

Dataset({
    features: ['fine_tuning_dataset'],
    num_rows: 65
})

In [72]:
#print(torch.cuda.memory_summary(device=None, abbreviated=False))

In [78]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
#     eval_dataset=val_dataset,
    peft_config=peft_config,
    dataset_text_field="fine_tuning_dataset",
    max_seq_length=2048,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

Map:   0%|          | 0/65 [00:00<?, ? examples/s]

In [ ]:
# Train model
trainer.train()

Step,Training Loss
5,0.860600
10,0.743100
15,0.542200
20,0.283000
25,0.184300
30,0.140500
35,0.161600
40,0.129100
45,0.145400
50,0.183200


In [ ]:
# Fine-tuned model name
#new_model_name = "mistral-ft-peft-v1-lr-64-with-more-data"

In [ ]:
# Save trained model
trainer.model.save_pretrained(new_model)

In [59]:
trainer.model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
              (v_proj)

In [15]:
from peft import PeftModel, PeftConfig

In [17]:
new_model_name = "/data/mistral/query-to-mql/exp-8/oct-30/checkpoint-2000"

In [15]:
#del model
# del trainer
torch.cuda.empty_cache()

In [16]:
nvsmi = nvidia_smi.getInstance()
nvsmi.DeviceQuery('memory.free, memory.total')

{'gpu': [{'fb_memory_usage': {'total': 16384.0,
    'free': 15972.9375,
    'unit': 'MiB'}}]}

In [18]:
# del model
torch.cuda.empty_cache()

from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(new_model_name, device_map="auto", torch_dtype=torch.bfloat16)
model = model.merge_and_unload()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [19]:
query_template_v1 = """Given the context : {context} and date reference: {date_input}, the query: {user_query}, is converted into below shown structured output.
[MQL]
"""

In [20]:
#model.to('cuda')

In [21]:
def predict_template_query_v1(user_query):
    inp = query_template_v1.format(context=context,
                                   user_query=user_query,
                                  date_input=date_input)
    _inputs = tokenizer.encode(inp, return_tensors="pt")
    outputs = model.generate(input_ids=_inputs.to('cuda'), max_length= 1700, pad_token_id=tokenizer.eos_token_id)
    output = tokenizer.decode(outputs[0])
    output_new = output.split('[MQL]\n')[1]
    return output_new.split('\n[/MQL]')[0], output
#     return output

In [22]:
%%time
user_query = 'show me the bottom 10 segments basis sales'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

user query:  show me the bottom 10 segments basis sales
----------------------------------------------------------------------------------------------------


/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:2507: UserWarning: Specified kernel cache directory could not be created! This disables kernel caching. Specified directory is /home/mosaic-ai/.cache/torch/kernels. This warning will appear only once per process. (Triggered internally at ../aten/src/ATen/native/cuda/jit_utils.cpp:1442.)
  next_tokens.tile(eos_token_id_tensor.shape[0], 1).ne(eos_token_id_tensor.unsqueeze(1)).prod(dim=0)


{'DIMENSION': {'segments': [{'ENTITY': 'Segment', 'RANK': [{'RANK ADJECTIVE': 'bottom', 'RANK VALUE': '10'}]}]}, 'MEASURE': {'sales': [{'ENTITY': 'Sales'}]}}
----------------------------------------------------------------------------------------------------
Step 1: Identify the components in the query
- "bottom 10 segments" - Rank adjective "bottom" and rank value "10"
- "basis sales" - Measure "Sales"

Step 2: Match the components to the context
- "bottom 10 segments" - Matches with "Segment" in the context under "DIMENSION" with the rank adjective "bottom" and rank value "10"
- "basis sales" - Matches with "Sales" in the context under "MEASURE"

Step 3: Convert the components into structured output
- "bottom 10 segments" - Create a "DIMENSION" object with "Segment" as the ENTITY and a "RANK" object containing the rank adjective "bottom" and rank value "10"
- "basis sales" - Create a "MEASURE" object with "Sales" as the ENTITY

Step 4: Check for date components
- The query does not h

In [23]:
%%time
user_query = 'top 2 and bottom 3 segments by sales'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

user query:  top 2 and bottom 3 segments by sales
----------------------------------------------------------------------------------------------------
{'DIMENSION': {'segments': [{'ENTITY': 'Segment', 'RANK': [{'RANK ADJECTIVE': 'top', 'RANK VALUE': '2'}, {'RANK ADJECTIVE': 'bottom', 'RANK VALUE': '3'}]}]}, 'MEASURE': {'sales': [{'ENTITY': 'Sales'}]}}
----------------------------------------------------------------------------------------------------
Step 1: Identify the components in the query
- "top 2": Rank adjective and rank value
- "bottom 3": Rank adjective and rank value
- "segments": Dimension (matches with 'Segment' in the context)
- "by sales": Measure (matches with 'Sales' in the context)

Step 2: Match the components to the context
- "segments" matches with 'Segment' in the context under 'DIMENSION'
- "sales" matches with 'Sales' in the context under 'MEASURE'

Step 3: Convert the query into structured output
- Create a 'DIMENSION' key and add 'segments' as an array with 'S

In [24]:
%%time
user_query = 'top 2 segments and bottom 3 sub-category basis quantity'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

user query:  top 2 segments and bottom 3 sub-category basis quantity
----------------------------------------------------------------------------------------------------
{'DIMENSION': {'segments': [{'ENTITY': 'Segment', 'RANK': [{'RANK ADJECTIVE': 'top', 'RANK VALUE': '2'}]}], 'sub-category': [{'ENTITY': 'Sub-Category', 'RANK': [{'RANK ADJECTIVE': 'bottom', 'RANK VALUE': '3'}]}]}, 'MEASURE': {'quantity': [{'ENTITY': 'Quantity'}]}}
----------------------------------------------------------------------------------------------------
Step 1: Identify the components in the query
- "top 2 segments": Rank adjective "top", rank value "2", and entity "segments"
- "bottom 3 sub-category": Rank adjective "bottom", rank value "3", and entity "sub-category"
- "basis quantity": Measure "quantity"

Step 2: Match the components to the context
- "segments" matches the entity "Segment" in the context with other names ['segment', 'segments', 'units', 'divisions'].
- "sub-category" matches the entity "Sub

In [25]:
%%time
user_query = 'quantity across segments except consumer and corporate in dubai'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

user query:  quantity across segments except consumer and corporate in dubai
----------------------------------------------------------------------------------------------------
{'DIMENSION': {'segments': [{'ENTITY': 'Segment'}]}, 'FILTER': {'consumer': [{'ENTITY': 'Consumer', 'EXCLUDE': 'True', 'PARENT': 'Segment'}], 'corporate': [{'ENTITY': 'Corporate', 'EXCLUDE': 'True', 'PARENT': 'Segment'}], 'dubai': [{'ENTITY': 'Dubai', 'PARENT': 'Country'}]}, 'MEASURE': {'quantity': [{'ENTITY': 'Quantity'}]}}
----------------------------------------------------------------------------------------------------
Step 1: Identify the components in the query
- Quantity: Measure
- Segments: Dimension
- Consumer: Filter (to be excluded)
- Corporate: Filter (to be excluded)
- Dubai: Filter

Step 2: Match the components to the context
- Quantity: Matches with 'Quantity' in the context
- Segments: Matches with 'Segment' in the context
- Consumer: Matches with 'Consumer' in the context (parent: Segment)
- C

In [26]:
%%time
user_query = 'in corporate share of phone and binder basis discount'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

user query:  in corporate share of phone and binder basis discount
----------------------------------------------------------------------------------------------------
{'DERIVED MEASURE': {'share': [{'APPLIED MEASURE': [{'discount': 'Discount'}], 'ENTITY': 'Ratio', 'RATIO FILTER': ['Phone', 'Binder']}]}, 'FILTER': {'binder': [{'ENTITY': 'Binder', 'PARENT': 'Sub-Category'}], 'corporate': [{'ENTITY': 'Corporate', 'PARENT': 'Segment'}], 'phone': [{'ENTITY': 'Phone', 'PARENT': 'Sub-Category'}]}, 'MEASURE': {'discount': [{'ENTITY': 'Discount'}]}}
----------------------------------------------------------------------------------------------------
Step 1: Identify the components in the query
- Corporate: Segment
- Share: Derived Measure (Ratio)
- Phone and Binder: Sub-Category
- Discount: Measure

Step 2: Match the components to the context
- Corporate: {'ENTITY': 'Corporate', 'other names': ['corporates', 'corporate'], 'parent': 'Segment'}
- Share: {'ENTITY': 'Ratio', 'other names': ['ratio'

In [27]:
%%time
user_query = 'list of under performing segments'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

user query:  list of under performing segments
----------------------------------------------------------------------------------------------------
{'DIMENSION': {'segments': [{'ADJECTIVE': ['under performing'], 'ENTITY': 'Segment', 'RANK': [{'RANK ADJECTIVE': 'bottom', 'RANK VALUE': '1'}], 'TONE': 'negative'}]}}
----------------------------------------------------------------------------------------------------
Step 1: Identify the main components of the query
- The query asks for a "list of under performing segments."

Step 2: Match the components to the context
- In the context, we have a 'DIMENSION' with an 'ENTITY' called 'Segment' and its other names as ['segment', 'segments', 'units', 'divisions'].

Step 3: Identify the adjectives and tone
- The query has an adjective "under performing" which indicates a negative tone.

Step 4: Identify the ranking
- The query asks for a list, which implies a ranking. Since it's asking for under performing segments, we can assume it's looking fo

In [28]:
%%time
user_query = 'when was the first time sales of segments was 0'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

user query:  when was the first time sales of segments was 0
----------------------------------------------------------------------------------------------------
{'DATE VARIABLE': {'when': [{'CONVERTED TIME ELEMENT': 'when', 'ENTITY': 'Order Date'}]}, 'DIMENSION': {'segments': [{'ENTITY': 'Segment'}]}, 'MEASURE': {'sales': [{'ENTITY': 'Sales', 'MEASURE CONSTRAINT': [{'COMPARISON VALUE': '0', 'COMPARSION OPERATOR': '='}]}]}}
----------------------------------------------------------------------------------------------------
Step 1: Identify the components in the query
- The query asks for "when" which indicates a date variable.
- The query mentions "sales" which is a measure.
- The query mentions "segments" which is a dimension.
- The query specifies a condition for sales to be "0".

Step 2: Match the components to the context
- "when" can be matched to the "Order Date" entity in the DATE VARIABLE context.
- "sales" can be matched to the "Sales" entity in the MEASURE context.
- "segment

In [29]:
%%time
user_query = 'sales of segments from beginning'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

user query:  sales of segments from beginning
----------------------------------------------------------------------------------------------------
{'DATE VARIABLE': {'beginning': [{'CONVERTED TIME ELEMENT': 'beginning', 'DATE RANGE': '01/01/2020 - 15/09/2023', 'ENTITY': 'Order Date'}]}, 'DIMENSION': {'segments': [{'ENTITY': 'Segment'}]}, 'MEASURE': {'sales': [{'ENTITY': 'Sales'}]}}
----------------------------------------------------------------------------------------------------
Step 1: Identify the components in the query
- The query mentions "sales" which can be matched to the "Sales" entity in the MEASURE context.
- The query mentions "segments" which can be matched to the "Segment" entity in the DIMENSION context.
- The query mentions "from beginning" which indicates a date range starting from the earliest date available.

Step 2: Match the components to the context
- "sales" is matched to the "Sales" entity in the MEASURE context.
- "segments" is matched to the "Segment" entity 

In [30]:
%%time
user_query = 'sales in 20/01/2020'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

user query:  sales in 20/01/2020
----------------------------------------------------------------------------------------------------
{'DATE VARIABLE': {'20/01/2020': [{'CONVERTED TIME ELEMENT': 'january 20, 2020', 'DATE RANGE': '2020/01/20 - 2020/01/20', 'ENTITY': 'Order Date'}]}, 'MEASURE': {'sales': [{'ENTITY': 'Sales'}]}}
----------------------------------------------------------------------------------------------------
Step 1: Identify the components in the query
- The query mentions "sales" which is a measure.
- The query also mentions "20/01/2020" which is a date component.

Step 2: Match the components to the context
- In the context, "sales" can be matched to the "Sales" entity under the "MEASURE" category.
- The date component "20/01/2020" needs to be converted to a proper date range.

Step 3: Convert the date component
- The date component "20/01/2020" refers to January 20, 2020.
- To convert it to a date range, we can use the format "YYYY/MM/DD". So, the date range for thi

In [31]:
%%time
user_query = 'sales in last one and half years'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

user query:  sales in last one and half years
----------------------------------------------------------------------------------------------------
{'DATE VARIABLE': {'last one and half years': [{'CONVERTED TIME ELEMENT': 'last one and half years', 'DATE RANGE': '2022/03/15 - 2023/09/15', 'ENTITY': 'Order Date'}]}, 'MEASURE': {'sales': [{'ENTITY': 'Sales'}]}}
----------------------------------------------------------------------------------------------------
Step 1: Identify the components in the query
- The query mentions "sales" which is a measure.
- The query also mentions "last one and half years" which is a date variable.

Step 2: Match the components to the context
- "sales" can be matched to the "Sales" entity in the context under MEASURE.
- "last one and half years" can be matched to the "Order Date" entity in the context under DATE VARIABLE.

Step 3: Convert the date component
- The date component "last one and half years" needs to be converted into a date range.
- To do this, 

In [32]:
%%time
user_query = 'trend of sales in dubai now'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

user query:  trend of sales in dubai now
----------------------------------------------------------------------------------------------------
{'DATE VARIABLE': {'trend': [{'CONVERTED TIME ELEMENT': 'now', 'DATE RANGE': '2023/09/15 - 2023/09/15', 'ENTITY': 'Order Date'}]}, 'FILTER': {'dubai': [{'ENTITY': 'Dubai', 'PARENT': 'Country'}]}, 'MEASURE': {'sales': [{'ENTITY': 'Sales'}]}}
----------------------------------------------------------------------------------------------------
Step 1: Identify components in the query
- The query mentions "trend of sales" which refers to the measure 'Sales' and the date variable 'Order Date'.
- The query also mentions "in Dubai" which refers to the filter 'Dubai'.

Step 2: Match components to the context
- 'Sales' can be matched to the measure 'Sales' in the context.
- 'Order Date' can be matched to the date variable 'Order Date' in the context.
- 'Dubai' can be matched to the filter 'Dubai' in the context.

Step 3: Convert date components
- The query

In [33]:
%%time
user_query = 'sales in the week of 01 March 2022'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

user query:  sales in the week of 01 March 2022
----------------------------------------------------------------------------------------------------
{'DATE VARIABLE': {'week of 01 March 2022': [{'CONVERTED TIME ELEMENT': 'week of 1 March 2022', 'DATE RANGE': '2022/02/28 - 2022/03/06', 'ENTITY': 'Order Date'}]}, 'MEASURE': {'sales': [{'ENTITY': 'Sales'}]}}
----------------------------------------------------------------------------------------------------
Step 1: Identify components in the query
- The query mentions "sales" which is a measure.
- The query also mentions "week of 01 March 2022" which is a date component.

Step 2: Match components to the context
- In the context, "sales" can be matched to the "Sales" entity under the "MEASURE" category.
- The date component "week of 01 March 2022" needs to be converted to a date range.

Step 3: Convert date component
- The date component "week of 01 March 2022" refers to the week starting from 1st March 2022. 
- To find the date range for 

In [34]:
%%time
user_query = 'top most selling segments in 2020 vs 2021'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

user query:  top most selling segments in 2020 vs 2021
----------------------------------------------------------------------------------------------------
{'DATE VARIABLE': {'2020 vs 2021': [{'CONVERTED TIME ELEMENT': '2020 vs 2021', 'DATE RANGE': '2020/01/01 - 2020/12/31, 2021/01/01 - 2021/12/31', 'ENTITY': 'Order Date'}]}, 'DIMENSION': {'segments': [{'ENTITY': 'Segment', 'RANK': [{'RANK ADJECTIVE': 'top', 'RANK VALUE': '1'}]}]}, 'MEASURE': {'selling': [{'ENTITY': 'Sales'}]}}
----------------------------------------------------------------------------------------------------
Step 1: Identify the components in the query
- "top most selling": Rank adjective "top" and Measure "Sales"
- "segments": Dimension "Segment"
- "2020 vs 2021": Date Variable "Order Date" with a comparison between two time periods

Step 2: Match the components to the context
- "top most selling" matches with the Measure "Sales"
- "segments" matches with the Dimension "Segment"
- "2020 vs 2021" matches with the Dat

In [35]:
%%time
user_query = 'sales from 100 to 200'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

user query:  sales from 100 to 200
----------------------------------------------------------------------------------------------------
{'MEASURE': {'sales': [{'ENTITY': 'Sales', 'MEASURE CONSTRAINT': [{'COMPARISON VALUE': '100', 'COMPARSION OPERATOR': '>='}, {'COMPARISON VALUE': '200', 'COMPARSION OPERATOR': '<='}]}]}}
----------------------------------------------------------------------------------------------------
Step 1: Identify the components in the query
- The query mentions "sales" which is a measure.
- The query also mentions a range "from 100 to 200" which is a constraint on the sales measure.

Step 2: Match the components to the context
- In the context, we can find the "Sales" entity under the 'MEASURE' category with other names like 'sales' and 'sale'.
- There is no date component mentioned in the query, so the date reference is not utilized here.

Step 3: Convert the query into a structured output
- Since the query is about sales and has a range from 100 to 200, we can 

In [36]:
%%time
user_query = 'sales from 2021 to 2023'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

user query:  sales from 2021 to 2023
----------------------------------------------------------------------------------------------------
{'DATE VARIABLE': {'sales': [{'CONVERTED TIME ELEMENT': 'from 2021 to 2023', 'DATE RANGE': '2021/01/01 - 2023/09/15', 'ENTITY': 'Order Date'}]}, 'MEASURE': {'sales': [{'ENTITY': 'Sales'}]}}
----------------------------------------------------------------------------------------------------
Step 1: Identify the components in the query
- The query mentions "sales" which is a measure.
- The query also mentions a date range "from 2021 to 2023" which is a date variable.

Step 2: Match the components to the context
- In the context, "sales" can be matched to the "Sales" entity under the "MEASURE" category.
- The date range "from 2021 to 2023" needs to be converted to a date range.

Step 3: Convert the date range
- The date range "from 2021 to 2023" indicates that we want to include all dates between 2021-01-01 and 2023-09-15.
- The date reference provided 

In [37]:
%%time
user_query = 'what is purchase across segments'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

user query:  what is purchase across segments
----------------------------------------------------------------------------------------------------
{'DIMENSION': {'segments': [{'ENTITY': 'Segment', 'RANK': [{'RANK ADJECTIVE': '', 'RANK VALUE': ''}]}]}, 'MEASURE': {'purchase': [{'ENTITY': 'Purchase Vol', 'MEASURE CONSTRAINT': [{'COMPARISON VALUE': '', 'COMPARSION OPERATOR': ''}]}]}}
----------------------------------------------------------------------------------------------------
Step 1: Identify the components in the query
- The query asks for "purchase" which can be matched to the context as "Purchase Vol".
- The query mentions "across segments" which can be matched to the context as "Segment".

Step 2: Match the components to the context
- "Purchase Vol" is a measure in the context, so it will be placed under the "MEASURE" section in the structured output.
- "Segment" is a dimension in the context, so it will be placed under the "DIMENSION" section in the structured output.

Step 3:

In [38]:
%%time
user_query = 'discount rate of phone and binders'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

user query:  discount rate of phone and binders
----------------------------------------------------------------------------------------------------
{'FILTER': {'binders': [{'ENTITY': 'Binder', 'PARENT': 'Sub-Category'}], 'phone': [{'ENTITY': 'Phone', 'PARENT': 'Sub-Category'}]}, 'MEASURE': {'discount rate': [{'ENTITY': 'Discount', 'MEASURE CONSTRAINT': [{'COMPARISON VALUE': '', 'COMPARSION OPERATOR': ''}]}]}}
----------------------------------------------------------------------------------------------------
Step 1: Identify components in the query
- The query mentions "discount rate" which is a measure.
- The query also mentions "phone" and "binders" which are filters.

Step 2: Match components to the context
- "discount rate" can be matched to the "Discount" entity in the context.
- "phone" can be matched to the "Phone" entity in the context, which has a parent "Sub-Category".
- "binders" can be matched to the "Binder" entity in the context, which also has a parent "Sub-Category".



In [39]:
%%time
user_query = 'discount rate of overall sub-category in corporate'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

user query:  discount rate of overall sub-category in corporate
----------------------------------------------------------------------------------------------------
{'DIMENSION': {'sub-category': [{'ENTITY': 'Sub-Category', 'RANK': [{'RANK ADJECTIVE': '', 'RANK VALUE': ''}]}]}, 'FILTER': {'corporate': [{'ENTITY': 'Corporate', 'PARENT': 'Segment'}]}, 'MEASURE': {'discount rate': [{'ENTITY': 'Discount', 'MEASURE CONSTRAINT': [{'COMPARISON VALUE': '', 'COMPARSION OPERATOR': ''}]}]}}
----------------------------------------------------------------------------------------------------
Step 1: Identify the components in the query
- The query mentions "discount rate" which is a measure.
- It also mentions "overall sub-category" which is a dimension.
- The query specifies "corporate" as a filter.

Step 2: Match the components to the context
- "discount rate" can be matched to the "Discount" entity in the context.
- "overall sub-category" can be matched to the "Sub-Category" entity in the contex

In [40]:
%%time
user_query = 'maximum sales of phone for consumer segment'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

user query:  maximum sales of phone for consumer segment
----------------------------------------------------------------------------------------------------
{'DIMENSION': {'segment': [{'ENTITY': 'Segment', 'RANK': [{'RANK ADJECTIVE': '', 'RANK VALUE': ''}]}]}, 'FILTER': {'consumer': [{'ENTITY': 'Consumer', 'PARENT': 'Segment'}], 'phone': [{'ENTITY': 'Phone', 'PARENT': 'Sub-Category'}]}, 'MEASURE': {'sales': [{'ADJECTIVE': ['maximum'], 'ENTITY': 'Sales', 'MEASURE CONSTRAINT': [{'COMPARISON VALUE': '', 'COMPARSION OPERATOR': ''}], 'TONE': 'positive'}]}}
----------------------------------------------------------------------------------------------------
Step 1: Identify the components in the query
- "maximum sales": Measure (Sales) with an adjective (maximum)
- "phone": Filter (Phone)
- "consumer segment": Filter (Consumer)

Step 2: Match the components to the context
- "maximum sales": Matched to "Sales" in the context with the adjective "maximum"
- "phone": Matched to "Phone" in the co

In [41]:
%%time
user_query = 'forecast of sales'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

user query:  forecast of sales
----------------------------------------------------------------------------------------------------
{'MEASURE': {'sales': [{'ENTITY': 'Sales', 'MEASURE CONSTRAINT': [{'COMPARISON VALUE': '', 'COMPARSION OPERATOR': ''}]}]}}
----------------------------------------------------------------------------------------------------
Step 1: Identify the components in the query
- The query is "forecast of sales". The main component here is "sales".

Step 2: Match the components to the context
- In the context, we can find "Sales" under the "MEASURE" category with other names such as "sales" and "sale". So, we can match the "sales" component in the query to the "Sales" entity in the context.

Step 3: Identify the date component in the query
- The query does not have any explicit date component mentioned. However, since it is a forecast, we can assume that the forecast is for the future period after the end_date mentioned in the date reference.

Step 4: Utilize the da

In [42]:
%%time
user_query = 'quantity across segments except consumer'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

user query:  quantity across segments except consumer
----------------------------------------------------------------------------------------------------
{'DIMENSION': {'segments': [{'ENTITY': 'Segment', 'RANK': [{'RANK ADJECTIVE': '', 'RANK VALUE': ''}]}]}, 'FILTER': {'consumer': [{'ENTITY': 'Consumer', 'EXCLUDE': 'True', 'PARENT': 'Segment'}]}, 'MEASURE': {'quantity': [{'ENTITY': 'Quantity', 'MEASURE CONSTRAINT': [{'COMPARISON VALUE': '', 'COMPARSION OPERATOR': ''}]}]}}
----------------------------------------------------------------------------------------------------
Step 1: Identify the components in the query
- The query asks for "quantity across segments except consumer". 
- The main components are "quantity" and "segments", with an exclusion of "consumer".

Step 2: Match the components to the context
- "quantity" can be matched to the "Quantity" entity in the MEASURE context.
- "segments" can be matched to the "Segment" entity in the DIMENSION context.
- "consumer" can be matc

In [43]:
%%time
user_query = 'which segment has highest purchase'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

user query:  which segment has highest purchase
----------------------------------------------------------------------------------------------------
{'DIMENSION': {'segment': [{'ENTITY': 'Segment', 'RANK': [{'RANK ADJECTIVE': 'top', 'RANK VALUE': '1'}]}]}, 'MEASURE': {'purchase': [{'ADJECTIVE': ['highest'], 'ENTITY': 'Purchase Vol', 'MEASURE CONSTRAINT': [{'COMPARISON VALUE': '', 'COMPARSION OPERATOR': ''}], 'TONE': 'positive'}]}}
----------------------------------------------------------------------------------------------------
Step 1: Identify the components in the query
- The query asks for the segment with the highest purchase. 
- Components: segment, highest, purchase

Step 2: Match the components to the context
- Segment: Found in the context under DIMENSION with the ENTITY "Segment"
- Purchase: Found in the context under MEASURE with the ENTITY "Purchase Vol"
- Highest: This is an adjective that indicates a ranking or comparison.

Step 3: Convert the query into structured outpu

In [44]:
%%time
user_query = 'top 2 segments basis discount'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

user query:  top 2 segments basis discount
----------------------------------------------------------------------------------------------------
{'DIMENSION': {'segments': [{'ENTITY': 'Segment', 'RANK': [{'RANK ADJECTIVE': 'top', 'RANK VALUE': '2'}]}]}, 'MEASURE': {'discount': [{'ENTITY': 'Discount'}]}}
----------------------------------------------------------------------------------------------------
Step 1: Identify the components in the query
- The query mentions "top 2 segments" which indicates a ranking and a dimension.
- The query also mentions "basis discount" which indicates a measure.

Step 2: Match the components to the context
- In the context, "segments" can be matched to the "Segment" entity under the "DIMENSION" category.
- "discount" can be matched to the "Discount" entity under the "MEASURE" category.

Step 3: Convert the query components into structured output
- For the dimension "segments", we need to include the "ENTITY" as "Segment" and the ranking information as "R

In [45]:
%%time
user_query = 'binder share of discount by consumer'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

user query:  binder share of discount by consumer
----------------------------------------------------------------------------------------------------


SyntaxError: closing parenthesis '}' does not match opening parenthesis '[' on line 10 (<string>, line 20)

In [46]:
%%time
user_query = 'binder and phone share basis sales for corporate'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

user query:  binder and phone share basis sales for corporate
----------------------------------------------------------------------------------------------------
{'DERIVED MEASURE': {'share': [{'APPLIED MEASURE': [{'sales': 'Sales'}], 'DERIVED MEASURE CONSTRAINT': [{'COMPARISON VALUE': '', 'COMPARSION OPERATOR': ''}], 'ENTITY': 'Ratio', 'RATIO FILTER': ['Binder', 'Phone']}]}, 'FILTER': {'binder': [{'ENTITY': 'Binder', 'PARENT': 'Sub-Category'}], 'corporate': [{'ENTITY': 'Corporate', 'PARENT': 'Segment'}], 'phone': [{'ENTITY': 'Phone', 'PARENT': 'Sub-Category'}]}}
----------------------------------------------------------------------------------------------------
Step 1: Identify the components in the query
- Binder
- Phone
- Share
- Sales
- Corporate

Step 2: Match the components to the context
- Binder: Sub-Category (Filter)
- Phone: Sub-Category (Filter)
- Share: Ratio (Derived Measure)
- Sales: Sales (Measure)
- Corporate: Segment (Filter)

Step 3: Convert the query into structured

In [47]:
%%time
user_query = 'in corporate, share of phone and binder basis discount'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

user query:  in corporate, share of phone and binder basis discount
----------------------------------------------------------------------------------------------------
{'DERIVED MEASURE': {'share': [{'APPLIED MEASURE': [{'discount': 'Discount'}], 'DERIVED MEASURE CONSTRAINT': [{'COMPARISON VALUE': '', 'COMPARSION OPERATOR': ''}], 'ENTITY': 'Ratio', 'RATIO FILTER': ['Phone', 'Binder']}]}, 'FILTER': {'binder': [{'ENTITY': 'Binder', 'PARENT': 'Sub-Category'}], 'corporate': [{'ENTITY': 'Corporate', 'PARENT': 'Segment'}], 'phone': [{'ENTITY': 'Phone', 'PARENT': 'Sub-Category'}]}, 'MEASURE': {'discount': [{'ENTITY': 'Discount', 'MEASURE CONSTRAINT': [{'COMPARISON VALUE': '', 'COMPARSION OPERATOR': ''}]}]}}
----------------------------------------------------------------------------------------------------
Step 1: Identify the components in the query
- Corporate: Segment
- Share: Derived Measure (Ratio)
- Phone and Binder: Sub-Category
- Discount: Measure

Step 2: Match the components to the

In [48]:
%%time
user_query = 'phone to binder ratio of discount in corporate'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

user query:  phone to binder ratio of discount in corporate
----------------------------------------------------------------------------------------------------
{'DATE': {'': []}, 'DERIVED MEASURE': {'ratio': [{'APPLIED MEASURE': [{'discount': 'Discount'}], 'DERIVED MEASURE CONSTRAINT': [{'COMPARISON VALUE': '', 'COMPARSION OPERATOR': ''}], 'ENTITY': 'Ratio', 'RATIO FILTER': ['Phone', 'Binder']}]}, 'DIMENSION': {'corporate': [{'ENTITY': 'Corporate', 'RANK': [{'RANK ADJECTIVE': '', 'RANK VALUE': ''}]}]}, 'FILTER': {'binder': [{'ENTITY': 'Binder', 'PARENT': 'Sub-Category'}], 'phone': [{'ENTITY': 'Phone', 'PARENT': 'Sub-Category'}]}, 'MEASURE': {'discount': [{'ENTITY': 'Discount', 'MEASURE CONSTRAINT': [{'COMPARISON VALUE': '', 'COMPARSION OPERATOR': ''}]}]}}
----------------------------------------------------------------------------------------------------
Step 1: Identify the components in the query
- Phone
- Binder
- Ratio
- Discount
- Corporate

Step 2: Match the components to the co

In [ ]:
%%time
user_query = 'segments with discount rate greater than 100k'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

user query:  segments with discount rate greater than 100k
----------------------------------------------------------------------------------------------------


In [ ]:
%%time
user_query = 'category with discount rate greater than 10k and quantity less than 2k'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = 'why did discount of phones drop'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = 'correlation of sales and purchase for phone'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = 'what will be the sales in q1 24'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = 'sales in q1 and q2 2021'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = 'how many segments contributing to growth of sales in p3m vs pp'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = 'how has sales trended in first week of 2021'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = 'what will be sales in 1st 5 days of 2024'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = 'sales and purchase across yoy'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = 'sales and purchase during last one year'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = 'In q1 2023, what was sales'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = 'What are drivers of sales growth'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = 'why did discount of phones decrease'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = 'why did sales of phones drop'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = 'why did discount of phones drop'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = 'why discount of phones dropped'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = 'discount of phones dropped, why?'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "Phones least sold in 2021"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "Phones most sold in 2021"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "In 02/19, What was the sales of phone?"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "In Feb 2019, What was the sales of phone?"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "worst performing sub category in 2021"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "What was phone sales in jun '20'?"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "Growth rate of sales share of phone"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "which are the 5 top selling sub category by sales in 2023"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "how does the sales change for phone in the last year"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "what is sales of phone in q1 2023"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "what is sales of phone in q3"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "what is sales of phone in q1"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "what as sales of phone in last one and half years"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "what is sales of phone now"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "what is sales of phone in 1st quarter 2022"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "what is sales of phone in 1st quarter 2019"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "what is sales of phone in q1 2023"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "what is sales of phone in Feb 2023"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "what is sales of phone in Sep 2023"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "what is sales of phone in last one and half month"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "what is sales of phone in 01/2023"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "what is sales of phone in 01/22"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "what is sales of phone in last 2.5 years"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "what is sales of phone in last week"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
context = """{
    "MEASURE": [{"ENTITY": "Discount", "other names": ["discount", "discount rate", "discount value", "deduction"]},
                {"ENTITY": "Purchase Vol", "other names": ["purchase", "purchase value", "purchase model"]},
                {"ENTITY": "Quantity", "other names": ["quantity", "volume"]},
                {"ENTITY": "Sales", "other names": ["sales", "sale"]}],
    "DIMENSION": [{"ENTITY": "Sub-Category", "other names": ["sub-category", "sub category", "categories", "section"]},
                  {"ENTITY": "Segment", "other names": ["segment", "segments", "units", "divisions"]},
                  {"ENTITY": "Parts", "other names": ["parts", "part", "section", "divisions"]},
                  {"ENTITY": "Country", "other names": ["country", "countries"]}],
    "FILTER": [{"ENTITY": "Consumer", "other names": ["consumers", "consumer"], "parent": "Segment"},
               {"ENTITY": "Phone", "other names": ["phone", "phones", "mobile phones"], "parent": "Sub-Category"},
               {"ENTITY": "Binder", "other names": ["binders", "binder"], "parent": "Sub-Category"},
               {"ENTITY": "Corporate", "other names": ["corporates", "corporate"], "parent": "Segment"},
               {"ENTITY": "India", "other names": ["india"], "parent": "Country"},
               {"ENTITY": "Dubai", "other names": ["dubai"], "parent": "Country"}],
    "DERIVED MEASURE": [{"ENTITY": "Ratio",
             "other names": ["ratio", "share", "contribution", "percentage", "proportion", "contributing"]},
            {"ENTITY": "Why", "other names": ["why", "cause of", "reason for", "diagnose"]},
            {"ENTITY": "contribution_to_growth", "other names": ["contribution to growth", "growth", "grown"]},
            {"ENTITY": "kda_transactional", "other names": ["kda", "key drivers", "key driver", "drivers", "driver"]},
            {"ENTITY": "Growth Rate", "other names": ["growth rate", "growth", "grown"]},
            {"ENTITY": "correlation",
             "other names": ["associate", "associated", "association", "associations", "correlate", "correlated",
                             "correlation", "correlations", "relate", "related", "relation", "relations",
                             "relationship",
                             "relationships"]}
            ],
    "DATE VARIABLE": [{"ENTITY": "Order Date", "other names": ["order date", "date", "trend", "time", "when", "mom", "yoy"]}]
    }"""

In [ ]:
date_input = {
    "start_date": "01/08/2020",
    "end_date": "20/12/2023"
}

In [ ]:
%%time
user_query = "what is sales of phone from beginning"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "what is sales of phone in last one year"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "what is sales of phone in last 2 year"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "what is sales of phone in last two year"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "what is sales of phone in second month of last year"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "what is sales of phone in last five years"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "sales of segment in 2nd month of last year"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "quantity of binders in fifth month of this year"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "discount of binders this year"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "discount of phones in last four years"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "quantity of phones in feb 2021 vs mar 2023"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "quantity of binders in fifth month of last year"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "discount of phones in last 1 year"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "sales of phones in last four months"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "sales of segments current year"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "sales of segments this year"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "sales of segments last year"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "sales in the last 2 weeks"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "discount of segments in last three weeks"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "quantity of binders in 10th month of last year"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "what is sales of phone in 01/2023"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "sales in 02/22"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "discount of phones in 04/23"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "quantity of binders in 04/2022"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

In [ ]:
%%time
user_query = "discount of segments in 07/23"
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])